## 语言模型与简单的WordEmbedding

什么是语言模型?

语言模型刻画了一句话,它作为人话(语言),存在的概率

对于一句话
$$sentence = w_1,w_2,\cdots,w_n $$
我们要求的概率为:
$$
P(sentence) = P( w_1,w_2,\cdots,w_n)
$$

按照概率公式展开，其形式如下：
$$
\begin{array}{ll}
 P(sentence) & = P( w_1,w_2,\cdots,w_n) \\
            & = P(w_1) P(w_2 | w_1) P(w_3|w_2,w_1)  \cdots P(w_n | w_1, \cdots, w_{n-1})
\end{array}
$$

给定一个freestyle：

> 嘿 嘿

> 准备好了没有

> 你看这个面它又长又宽

> 就像这个碗它又大又圆

> 你们 来这里 吃饭

> 觉得 饭 很 好吃

> 我看行

> 你们 来这里 吃饭

> 我给你们拉面一样很开心

实际上，$P(freestyle)$是非常难算的

$$
\begin{array}{ll}
 P(w_1)  = P(\mbox{嘿})& = \frac{\#\mbox{嘿}}{\#\mbox{总词数}} \\
 p(w_2 | w_1) = P(\mbox{嘿} | \mbox{嘿})           & = \frac{\#\mbox{嘿,嘿}}{\#(\mbox{嘿},*)} \\
  p(w_3 | w_1, w_2) = P(\mbox{准备} | \mbox{嘿},\mbox{嘿})           & = \frac{\#\mbox{嘿 嘿 准备}}{\#(\mbox{嘿,嘿},*)} \\
    p(w_4 | w_1, w_2, w_3) = P(\mbox{好} | \mbox{嘿},\mbox{嘿}，\mbox{准备})           & = \frac{\#\mbox{嘿 嘿 准备 好}}{\#(\mbox{嘿 嘿 准备},*)} \\
    \cdots \\
\end{array}
$$

## N-gram 语言模型

为了简化$P(sentence)$， 人们强行设置了一个条件概率的依赖长度
- 没有依赖的， 0-gram
$$
\begin{array}{ll}
 P(sentence) & = P( w_1,w_2,\cdots,w_n) \\
            & = P(w_1) P(w_2 | w_1)  P(w_3|w_2,w_1)  \cdots  P(w_n | w_1, \cdots, w_{n-1}) \\
            & \approx P(w_1)  P(w_2)  \cdots  P(w_n) \\
            & = P(\mbox{嘿})  P(\mbox{嘿})  P(\mbox{准备})  P(\mbox{好})  \cdots  P(\mbox{开心})
\end{array}
$$
- 依赖为1的，就是1-gram
$$
\begin{array}{ll}
 P(sentence) & = P( w_1,w_2,\cdots,w_n) \\
            & = P(w_1)  P(w_2 | w_1)  P(w_3|w_2,w_1)  \cdots  P(w_n | w_1, \cdots, w_{n-1}) \\
            & \approx P(w_1 | start)  P(w_2 | w_1)   P(w_3 | w_2)\cdots  P(w_n|w_{n-1}) \\
            & = P(\mbox{嘿}| start)  P(\mbox{嘿} | \mbox{嘿})  P(\mbox{准备} | \mbox{嘿})  P(\mbox{好} | \mbox{准备})  \cdots  P(\mbox{开心} | \mbox{很})
\end{array}
$$
- 依赖为2的，就是2-gram
$$
 P(sentence) \approx P(\mbox{嘿}| start)  P(\mbox{嘿} | \mbox{嘿}, start)  P(\mbox{准备} | \mbox{嘿}, \mbox{嘿})  P(\mbox{好} | \mbox{准备}, \mbox{嘿})  \cdots  P(\mbox{开心} | \mbox{很}, \mbox{一样})
$$


In [1]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
test_sentence = """ 嘿 嘿
                    准备 好 了 没有
                    你 看 这个 面 它 又 长 又 宽
                    就 像 这个 碗 它 又 大 又 圆
                    你们 来 这里 吃饭
                    觉得 饭 很 好吃
                    我 看 行
                    你们 来 这里 吃饭
                    我 给 你们 拉 面 一样 很 开心""".split()

In [2]:
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2]) for i in range(len(test_sentence) - 2)]
print(trigrams[:10])

[(['嘿', '嘿'], '准备'), (['嘿', '准备'], '好'), (['准备', '好'], '了'), (['好', '了'], '没有'), (['了', '没有'], '你'), (['没有', '你'], '看'), (['你', '看'], '这个'), (['看', '这个'], '面'), (['这个', '面'], '它'), (['面', '它'], '又')]


In [4]:
vocab = set(test_sentence)
print(vocab)

{'你', '它', '圆', '宽', '看', '就', '觉得', '你们', '好吃', '一样', '嘿', '准备', '这里', '好', '很', '来', '给', '我', '面', '又', '开心', '拉', '吃饭', '了', '长', '碗', '没有', '行', '这个', '大', '像', '饭'}


In [5]:
word_to_id = {word: i for i, word in enumerate(vocab)}
id_to_word = {i: word for i, word in enumerate(vocab)}
print(word_to_id)

{'你': 0, '它': 1, '圆': 2, '宽': 3, '看': 4, '就': 5, '觉得': 6, '你们': 7, '好吃': 8, '一样': 9, '嘿': 10, '准备': 11, '这里': 12, '好': 13, '很': 14, '来': 15, '给': 16, '我': 17, '面': 18, '又': 19, '开心': 20, '拉': 21, '吃饭': 22, '了': 23, '长': 24, '碗': 25, '没有': 26, '行': 27, '这个': 28, '大': 29, '像': 30, '饭': 31}


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

$$ p(w_n | w_{n-1}, w_{n+1}) $$

In [9]:
class NgramLanguageModel(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NgramLanguageModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = self.linear1(embeds)
        out = F.relu(out)
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [10]:
loss_function = nn.NLLLoss()

In [11]:
model = NgramLanguageModel(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)

In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [14]:
trigrams

[(['嘿', '嘿'], '准备'),
 (['嘿', '准备'], '好'),
 (['准备', '好'], '了'),
 (['好', '了'], '没有'),
 (['了', '没有'], '你'),
 (['没有', '你'], '看'),
 (['你', '看'], '这个'),
 (['看', '这个'], '面'),
 (['这个', '面'], '它'),
 (['面', '它'], '又'),
 (['它', '又'], '长'),
 (['又', '长'], '又'),
 (['长', '又'], '宽'),
 (['又', '宽'], '就'),
 (['宽', '就'], '像'),
 (['就', '像'], '这个'),
 (['像', '这个'], '碗'),
 (['这个', '碗'], '它'),
 (['碗', '它'], '又'),
 (['它', '又'], '大'),
 (['又', '大'], '又'),
 (['大', '又'], '圆'),
 (['又', '圆'], '你们'),
 (['圆', '你们'], '来'),
 (['你们', '来'], '这里'),
 (['来', '这里'], '吃饭'),
 (['这里', '吃饭'], '觉得'),
 (['吃饭', '觉得'], '饭'),
 (['觉得', '饭'], '很'),
 (['饭', '很'], '好吃'),
 (['很', '好吃'], '我'),
 (['好吃', '我'], '看'),
 (['我', '看'], '行'),
 (['看', '行'], '你们'),
 (['行', '你们'], '来'),
 (['你们', '来'], '这里'),
 (['来', '这里'], '吃饭'),
 (['这里', '吃饭'], '我'),
 (['吃饭', '我'], '给'),
 (['我', '给'], '你们'),
 (['给', '你们'], '拉'),
 (['你们', '拉'], '面'),
 (['拉', '面'], '一样'),
 (['面', '一样'], '很'),
 (['一样', '很'], '开心')]

In [15]:
for epoch in range(1000):
    total_loss = 0
    for context, target in trigrams:
        context_tensor = torch.tensor([word_to_id[w] for w in context], dtype=torch.long)
        optimizer.zero_grad()
        log_probs = model(context_tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_id[target]], dtype=torch.long))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if epoch % 20 == 0:
        print(total_loss/len(vocab))

4.896181091666222
4.323531731963158
3.815610770136118
3.3711194321513176
2.979199256747961
2.621775332838297
2.2896604537963867
1.981300052255392
1.7002220638096333
1.4492377378046513
1.2299880422651768
1.0424759909510612
0.885625995695591
0.7568334415555
0.6524491757154465
0.5683880597352982
0.5009026676416397
0.4466545283794403
0.40271973609924316
0.36678197979927063
0.3370480537414551
0.31222742795944214
0.2912861406803131
0.2734541594982147
0.2581346482038498
0.24486128985881805
0.23327478766441345
0.22309532761573792
0.21409006416797638
0.20608000457286835
0.19892030954360962
0.19248811900615692
0.18667468428611755
0.18141070008277893
0.17662087082862854
0.17225471138954163
0.16825039684772491
0.16457854211330414
0.16119390726089478
0.15806061029434204
0.15516133606433868
0.15247492492198944
0.14998182654380798
0.1476566195487976
0.1454886645078659
0.14345882833003998
0.14156000316143036
0.1397707313299179
0.1380997747182846
0.13651655614376068


In [ ]:
test_sentence = """ 嘿 嘿
                    准备 好 了 没有
                    你 看 这个 面 它 又 长 又 宽
                    就 像 这个 碗 它 又 大 又 圆
                    你们 来 这里 吃饭
                    觉得 饭 很 好吃
                    我 看 行
                    你们 来 这里 吃饭
                    我 给 你们 拉 面 一样 很 开心""".split()

In [19]:
data = model(torch.tensor([word_to_id["一样"],word_to_id["很"]]))
log_prob, index = data.max(1)
id_to_word[index.item()]

'开心'

In [21]:
a = model.embeddings(torch.tensor(word_to_id["你们"]))
a

tensor([-0.9184,  0.3898, -1.5934,  1.3574, -1.2248,  0.0207, -1.2754,  0.6992,
         0.5606,  0.3963], grad_fn=<EmbeddingBackward>)

In [22]:
b = model.embeddings(torch.tensor(word_to_id["你"]))
b

tensor([ 1.3453, -0.4587, -0.4180,  0.2134, -0.2366, -0.5281,  0.2433, -1.5369,
        -0.2175, -0.1851], grad_fn=<EmbeddingBackward>)